<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Industrielles-Management/blob/main/ErweiterungNetworkDesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design Erweiterung um Beschaffungsmärkte

In [1]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [2]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Datenaufbereitung


1.   Problemrelevante Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/NetworkDesign

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [5]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/NetworkDesign && pwd

/content/drive/MyDrive/Industrielles_Management/Daten/NetworkDesign


In [6]:
# Daten laden
import pandas as pd

In [7]:
path = "drive/MyDrive/Industrielles_Management/Daten/NetworkDesign"

In [8]:
kapa=pd.read_csv(f"{path}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [9]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [10]:
produktionskosten=pd.read_csv(f"{path}/NetworkDesign_Produktionskosten.csv", sep=";")

In [11]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [12]:
nachfrage=pd.read_csv(f"{path}/NetworkDesign_Nachfrage.csv", sep=";")

In [13]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


In [14]:
beschaffung = pd.read_csv(f"{path}/NetworkDesign_Beschaffungskosten.csv", sep=";")

In [15]:
beschaffung.head()

,Beschaffungsmarkt,Vorprodukt,Beschaffungskosten,Kapazitäten
0,Afrika,Flasche,10,60
1,Afrika,Vitaminwasser,20,30
2,Asien,Flasche,12,100
3,Asien,Vitaminwasser,15,40
4,Europa,Flasche,15,60


## Indexmengen

In [16]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [17]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

In [18]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

In [19]:
M = beschaffung["Beschaffungsmarkt"].unique().tolist() # Menge der Beschaffungsmärkte

In [20]:
P = beschaffung["Vorprodukt"].unique().tolist() # Menge der Vorprodukte

## Entscheidungsvariablen

In [24]:
infinity = solver.infinity()

In [25]:
# Definition der Entscheidungsvariablen
# x: Fluss von Fertigprodukten vom Produktionsstandort in den Markt
x={}
for i in I:
  for j in J: 
    x[i,j] = solver.NumVar(0.0, infinity, f"{i},{j}")

In [26]:
# Definition der Entscheidungsvariablen
# y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
y={}
for i in I:
  for a in A: 
    y[i,a] = solver.BoolVar(f"{i},{a}")

In [27]:
# Definition der Entscheidungsvariablen
# z: Fluss von Vorprodukten vom Beschaffungsmarkt in den Produktionsstandort
z={}
for m in M:
  for p in P:
    for i in I: 
      z[m,p,i] = solver.NumVar(0.0, infinity, f"{m},{p},{i}")

In [28]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 85


## Parameter

In [29]:
# Herstell- und Transportkosten
produktionskosten.set_index(["Produktionsstandort","Markt"], inplace=True)

In [32]:
pc = produktionskosten.to_dict("dict")["Produktionskosten"]

In [33]:
pc

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71}

In [34]:
# Nachfrage aus den Märtken

In [35]:
nachfrage.set_index(["Markt"], inplace=True)

In [36]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [37]:
d

{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

In [38]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapa.set_index(["Produktionsstandort","Ausbaustufe"], inplace=True)

In [39]:
pcap = kapa.to_dict("dict")["Kapazitäten"]

In [40]:
pcap

{('Charleston', 'klein'): 10,
 ('Charleston', 'groß'): 20,
 ('Curitiba', 'klein'): 10,
 ('Curitiba', 'groß'): 20,
 ('Hamburg', 'klein'): 10,
 ('Hamburg', 'groß'): 20,
 ('Quanzhou', 'klein'): 10,
 ('Quanzhou', 'groß'): 20,
 ('East London', 'klein'): 10,
 ('East London', 'groß'): 20}

In [41]:
f = kapa.to_dict("dict")["Investitionen"]

In [42]:
f

{('Charleston', 'klein'): 6000,
 ('Charleston', 'groß'): 9000,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

In [43]:
# Beschaffungskosten und Kapazitäten
beschaffung.set_index(["Beschaffungsmarkt","Vorprodukt"], inplace=True)

In [44]:
bc = beschaffung.to_dict("dict")["Beschaffungskosten"]

In [45]:
bc

{('Afrika', 'Flasche'): 10,
 ('Afrika', 'Vitaminwasser'): 20,
 ('Asien', 'Flasche'): 12,
 ('Asien', 'Vitaminwasser'): 15,
 ('Europa', 'Flasche'): 15,
 ('Europa', 'Vitaminwasser'): 20,
 ('Südamerika', 'Flasche'): 8,
 ('Südamerika', 'Vitaminwasser'): 10,
 ('USA', 'Flasche'): 11,
 ('USA', 'Vitaminwasser'): 6}

In [46]:
bcap = beschaffung.to_dict("dict")["Kapazitäten"]

In [47]:
bcap

{('Afrika', 'Flasche'): 60,
 ('Afrika', 'Vitaminwasser'): 30,
 ('Asien', 'Flasche'): 100,
 ('Asien', 'Vitaminwasser'): 40,
 ('Europa', 'Flasche'): 60,
 ('Europa', 'Vitaminwasser'): 60,
 ('Südamerika', 'Flasche'): 40,
 ('Südamerika', 'Vitaminwasser'): 30,
 ('USA', 'Flasche'): 50,
 ('USA', 'Vitaminwasser'): 35}

## Zielfunktion

In [48]:
# Maximierung der gesamten Deckungsbeiträge
solver.Minimize(
    sum(pc[i,j]*x[i,j] for i in I for j in J) + #Produktionskosten
    sum(bc[m,p]*z[m,p,i] for m in M for p in P for i in I) + #Beschaffungskosten
    sum(f[i,a]*y[i,a] for i in I for a in A) #Invest
    )

## Nebenbedingungen

In [49]:
# Nachfrage in den Märkten erfüllen
for j in J:
  solver.Add(sum(x[i,j] for i in I)>=d[j])

# Kapazitäten an den Standorten aufbauen und einhalten
for i in I:
  solver.Add(sum(x[i,j] for j in J)<= sum(pcap[i,a]*y[i,a] for a in A))

# Kapazitäten der Beschaffungsmärkte einhalten
for m in M:
  for p in P:
    solver.Add(sum(z[m,p,i] for i in I)<= bcap[m,p])

# Flussbedingung
for p in P:
  for i in I:
    solver.Add(sum(z[m,p,i] for m in M) == sum(x[i,j] for j in J))

## Berechnung Lösung

In [50]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
    print("\n")
    print("Produktions-Markt-Zuteilung")
    for j in J:
      print(f"Markt:{j}")
      for i in I:
        if x[i,j].solution_value()>0:
          print(f'{i,j} =', round(x[i,j].solution_value()))
    print("\n")
    print("Invest Standorte")
    for i in I:
      for a in A:
        if y[i,a].solution_value()>0:
          print(f'{i,a} =', round(y[i,a].solution_value()))
    print("\n")
    print("Beschaffungsmengen")
    for i in I:
      for p in P:
        for m in M:
          if z[m,p,i].solution_value()>0:
            print(f'{i,p,m} =', round(z[m,p,i].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 24671.0


Produktions-Markt-Zuteilung
Markt:USA
('Curitiba', 'USA') = 12
Markt:Südamerika
('Curitiba', 'Südamerika') = 8
Markt:Europa
('Quanzhou', 'Europa') = 4
('East London', 'Europa') = 10
Markt:Asien
('Quanzhou', 'Asien') = 16
Markt:Afrika
('East London', 'Afrika') = 7


Invest Standorte
('Curitiba', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


Beschaffungsmengen
('Curitiba', 'Flasche', 'Südamerika') = 20
('Curitiba', 'Vitaminwasser', 'USA') = 20
('Quanzhou', 'Flasche', 'Afrika') = 17
('Quanzhou', 'Flasche', 'Südamerika') = 3
('Quanzhou', 'Vitaminwasser', 'Südamerika') = 5
('Quanzhou', 'Vitaminwasser', 'USA') = 15
('East London', 'Flasche', 'Südamerika') = 17
('East London', 'Vitaminwasser', 'Südamerika') = 17
